## Connect using psycopg2
----------------

In [22]:
import psycopg2

In [23]:
create_sql = '''
drop table if exists test.service_windows;
create table if not exists test.service_windows (
    id int, branch_id int, location_id int,
    service_window json
);'''
select_sql = '''
select * from test.service_windows
'''
insert_sql = '''
insert into test.service_windows (id, branch_id, location_id, service_window)
values (1,1,1,
'[
{
"sd": "M", 
"sw": ["0600-1200","1400-1500"]
},
{
"sd": "T", 
"sw": ["0600-1200","1400-1500"]}]');
insert into test.service_windows (id, branch_id, location_id, service_window)
values (2,2,2,
'[
{"sd": "W", "sw": ["0600-1200","1400-1500"]},
{"sd": "H", "sw": ["0600-1200"]}
]');
'''

In [73]:
with psycopg2.connect("dbname=brinks") as conn:
    with conn.cursor() as cur:
        cur.execute(create_sql)
        cur.execute(insert_sql)
        cur.execute(select_sql)
        for rec in cur:
            print(rec)

(1, 1, 1, 1, {'address': '111 Main'}, {'lat': 1.00001, 'long': 1.0}, 1, [{'sd': 'S', 'sw': ['00:01-23-59']}])
(2, 1, 2, 1, {'address': '222 Main'}, {'lat': 1.00002, 'long': 1.0}, 2, [{'sd': 'S', 'sw': ['00:01-23-59']}, {'sd': 'U', 'sw': ['00:01-23-59']}])
(3, 1, 3, 3, {'address': '333 Main'}, {'lat': 1.00003, 'long': 1.0}, 3, [{'sd': 'S', 'sw': ['00:01-23-59']}, {'sd': 'U', 'sw': ['00:01-23-59']}, {'sd': 'T', 'sw': ['00:09-10:00']}])


## Connect using SQLAlchemy
----------------------

In [13]:

from sqlalchemy import Table, Column, Integer, String, MetaData
from sqlalchemy import create_engine
from sqlalchemy.sql import text

db = create_engine("postgresql://localhost:5432/brinks")
conn = db.connect()

In [14]:
sql = text("select * from test.service_windows")
results = conn.execute(sql)

In [15]:
for row in results:
    print(row)

(1, 1, 1, 1, {'address': '111 Main'}, {'lat': 1.00001, 'long': 1.0}, 1, [{'sd': 'S', 'sw': ['00:01-23-59']}])
(2, 1, 2, 1, {'address': '222 Main'}, {'lat': 1.00002, 'long': 1.0}, 2, [{'sd': 'S', 'sw': ['00:01-23-59']}, {'sd': 'U', 'sw': ['00:01-23-59']}])
(3, 1, 3, 3, {'address': '333 Main'}, {'lat': 1.00003, 'long': 1.0}, 3, [{'sd': 'S', 'sw': ['00:01-23-59']}, {'sd': 'U', 'sw': ['00:01-23-59']}, {'sd': 'T', 'sw': ['00:09-10:00']}])


In [30]:
# test for creating table with sqlalchemy
metadata = MetaData()
books = Table(
    'book', metadata,
    Column('id', Integer, primary_key=True),
    Column('title', String),
    Column('primary_author', String),
    schema='test',
    autoload_replace=True,
)
metadata.create_all(db)

In [29]:
metadata.drop_all(db)
conn.close()